# GNN-in-the-Loop Slot Attention 训练

## 核心创新
在 Slot Attention 的每次迭代中插入 GNN 消息传递，让 slot 之间协作发现部件。

## 与原版的区别
- 原版：slot 独立竞争像素
- 我们：slot 通过 GNN 感知彼此，避免重叠和遗漏

In [ ]:
# 1. 检查 GPU
!nvidia-smi

In [ ]:
# 2. 克隆/更新代码
!git clone https://github.com/alltobebetter/PartGraph-FewShot.git 2>/dev/null || (cd PartGraph-FewShot && git pull)
%cd PartGraph-FewShot

In [ ]:
# 3. 安装依赖
!pip install -q -r requirements.txt

In [ ]:
# 4. 下载 CUB-200 数据集
!mkdir -p data
%cd data
!wget -q -N https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz
!tar -xzf CUB_200_2011.tgz 2>/dev/null || echo 'Already extracted'
%cd ..

In [ ]:
# 5. 训练 GNN-in-the-Loop 版本
# 注意：batch_size=8 防止 OOM，slot_dim=128 减少显存
!python src/train_gnn_slot.py \
    --data_root ./data \
    --output_dir ./checkpoints_gnn_slot \
    --batch_size 8 \
    --epochs 30 \
    --num_slots 8 \
    --slot_dim 128 \
    --slot_iters 3 \
    --gnn_start_iter 1 \
    --lambda_cls 1.0 \
    --lambda_recon 0.1 \
    --lambda_div 0.01 \
    --lambda_entropy 0.001

In [ ]:
# 6. Few-Shot 评估 (GNN 版本)
!python src/eval_fewshot_gnn.py \
    --data_root ./data \
    --checkpoint ./checkpoints_gnn_slot/best_model.pth \
    --use_gnn \
    --num_slots 8 \
    --slot_dim 128 \
    --slot_iters 3 \
    --gnn_start_iter 1 \
    --n_way 5 \
    --k_shot 5 \
    --n_episodes 200

In [ ]:
# 7. 查看可视化结果
from IPython.display import Image, display
import glob

vis_files = sorted(glob.glob('./checkpoints_gnn_slot/epoch_*_vis.png'))
for f in vis_files[-5:]:
    print(f)
    display(Image(filename=f))

## 消融实验：对比有无 GNN

如果时间允许，可以跑一个 baseline 对比

In [ ]:
# 8. (可选) 训练 Baseline (无 GNN)
!python src/train.py \
    --data_root ./data \
    --output_dir ./checkpoints_baseline \
    --batch_size 8 \
    --epochs 30 \
    --num_slots 8 \
    --slot_dim 128

In [ ]:
# 9. (可选) 评估 Baseline
!python src/eval_fewshot_gnn.py \
    --data_root ./data \
    --checkpoint ./checkpoints_baseline/best_model.pth \
    --num_slots 8 \
    --slot_dim 128 \
    --n_way 5 \
    --k_shot 5 \
    --n_episodes 200